# PROCESSO DE CRIAÇÃO DO MODELO DE MACHINE LEARNING PARA PREVER A PRESENÇA DE MINÉRIOS EM DETERMINADA COORDENADA

O conteúdo descrito neste notebook visa desenvolver um modelo capaz de prever a presença de minérios em coordenadas geográficas específicas. Este processo envolve a coleta de dados presentes no banco de dados do Mineral Resources Data System (https://mrdata.usgs.gov/mrds/) e a manipulação afim de criar um modelo de machine leraning que pode ser usado para fazer previsões sobre a presença de minérios em novas coordenadas. Esse tópico é de grande importância na indústria de mineração, pois pode ajudar a otimizar a exploração de recursos naturais e a tomada de decisões estratégicas.

## 1. PREPARAÇÃO INICIAL

### 1.1. Importação de libs

In [ ]:
import os, wget, zipfile

import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import random

import pickle

### 1.2. Carregando dados

In [ ]:
# Downloading Data
data_url = 'https://mrdata.usgs.gov/mrds/mrds-csv.zip'
wget.download(data_url)
z = zipfile.ZipFile('mrds-csv.zip')
z.extract('mrds.csv')
z.close()
os.remove('mrds-csv.zip')

In [ ]:
df = pd.read_csv('mrds.csv')

In [ ]:
df['score'].value_counts()

## 2. ANÁLISE EXPLORATÓRIA

### 2.1. Visualizando topo do dataframe

In [ ]:
df.head()

### 2.3. Informações sobre o dataframe

In [ ]:
df.info()

## 3. PREPROCESSAMENTO DOS DADOS PARA ML

### 3.1. Removendo colunas inutilizadas

In [ ]:
df = df[['latitude', 'longitude', 'ore']]
df

### 3.2. Removendo linhas contendo valor nulo

In [ ]:
df = df.dropna(subset=['ore', 'latitude', 'longitude'])
df

### 3.3. Separando conteúdo da coluna 'ore' em novas colunas

#### 3.3.1. Transformando items do dataframe separados por vírgula em uma lista de items

In [ ]:
df['ore'] = df['ore'].str.split(',').apply(lambda x: [item.strip() for item in x] if isinstance(x, list) else x)
df

#### 3.3.2. Transformando os itens da coluna ore
Transformando os itens da coluna ore para novas colunas num dataframe temporário, onde seu conteúdo é do tipo bool, correspondendo à presença do minério na linha correspondente.

In [ ]:
ores = []

for ore_list in df['ore']:
    for ore in ore_list:
        if ore not in ores:
            ores.append(ore)

ore_dummies = df['ore'].apply(lambda x: pd.Series([1 if ore in x else 0 for ore in ores]))

columns = {}
counter = 0
for ore in ores:
    columns[counter] = ore
    counter += 1

ore_dummies = ore_dummies.rename(columns=columns)
ore_dummies

#### 3.3.3. Selecionando os 30 minerais com maior contagem de registros
Selecionando os 30 minerais com maior contagem de registros em virtude da existência de minerais com baixa contagem.

In [ ]:
ores_zip = []

for ore in ores:
    value_counts = ore_dummies[ore].value_counts()
    ores_zip.append((ore, value_counts[1]))

sorted_zips = sorted(ores_zip, key=lambda x: x[1], reverse=True)
top_10_minerais = sorted_zips[:30]
top_10_minerais

#### 3.3.4. Descartando minerais não selecionados

In [ ]:
drop_ores = []

for x in sorted_zips[30:]:
    drop_ores.append(x[0])
    
ore_dummies = ore_dummies.drop(drop_ores, axis=1)
ore_dummies

#### 3.3.5. Concatenando dataframes

Concatenando dataframe original com o dataframe temporário e atribuindo a um novo dataframe, descartando a coluna 'ore'

In [ ]:
new_df = pd.concat([df, ore_dummies], axis=1)
new_df = new_df.drop(['ore'], axis=1)
new_df

## 4. CRIANDO OS MODELOS DE ML

### 4.1. Criando um modelo de ML para cada ore
Criando um modelo de ML para cada ore e exibindo seus scores correspondentes

In [ ]:
models = {}

for column in new_df.drop(['latitude', 'longitude'], axis=1).columns:
    
    print(f'Training model for "{column}"')
    
    y = new_df[column]
    X = new_df[['latitude', 'longitude']]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
    
    rf_model = RandomForestClassifier()
    rf_model.fit(X_train, y_train)
    
    print(f'Scores: Training = {rf_model.score(X_train, y_train)} - Test = {rf_model.score(X_test, y_test)}\n')
    
    models[column] = rf_model

### 4.2. Salvando modelos criados na pasta models

In [ ]:
for ore in models:
    path = 'models/'
    fname = f'{ore.lower()}.pickle'
    with open(path+fname, 'wb') as f:
        pickle.dump(models[ore], f)
        print(f'\"{fname}\" saved')

### 4.3. Exemplo: consumindo modelo através do arquivo para gerar pins

In [ ]:
# Caminho para o modelo salvo. Neste exemplo, é usado o 'gold.pickle'.
model_fpath = 'models/gold.pickle'

# Número de Pins a serem criados.
n_pins = 5

def get_pins(model_fpath, n_pins):

    # Constantes Imutáveis
    MIN_LATITUDE = -90
    MAX_LATITUDE = 90
    MIN_LONGITUDE = -180
    MAX_LONGITUDE = 180

    pins = []

    with open(model_fpath, 'rb') as f:

        rf = pickle.load(f)

        while (len(pins)<n_pins):

            random_data = {
                'latitude': [random.uniform(MIN_LATITUDE, MAX_LATITUDE)],
                'longitude': [random.uniform(MIN_LONGITUDE, MAX_LONGITUDE)]
            }

            predict_X = pd.DataFrame(random_data)
            predict = rf.predict(predict_X)

            if predict[0] == 1:
                pins.append({'latitude': predict_X['latitude'][0], 'longitude': predict_X['longitude'][0]})

    return pins

get_pins(model_fpath, n_pins)